In [13]:
import re
import tweepy
import json
from tweepy import OAuthHandler
from textblob import TextBlob

In [14]:
class TwitterClient(object): 
    def __init__(self):
        with open('api.json') as f:
            api = json.load(f)
        consumer_key = api["Key"]
        consumer_secret = api["Secret"]
        access_token = api["Access Token"]
        access_token_secret = api["Access Token Secret"]
  
        try: 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            self.auth.set_access_token(access_token, access_token_secret) 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 
  
    def clean_tweet(self, tweet): 
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

    def get_tweet_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
  
    def get_tweets(self, query, count = 10):
        tweets = []
        try:
            fetched_tweets = self.api.search(q = query, count = count)
            for tweet in fetched_tweets:
                parsed_tweet = {}
                parsed_tweet['text'] = tweet.text
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
                if tweet.retweet_count > 0:
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
            return tweets
        except tweepy.TweepError as e:
            print("Error : " + str(e))

In [22]:
def main():
    api = TwitterClient()
    tweets = api.get_tweets(query = 'Donald Trump', count = 200)
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    print("Positive tweets percentage: " + str(round(100*len(ptweets)/len(tweets), 2)) + "%")
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    print("Negative tweets percentage: "+ str(round(100*len(ntweets)/len(tweets), 2)) + "%")
    print("Neutral tweets percentage: " + str(round(100*(len(tweets)-len(ptweets)-len(ntweets))/len(tweets), 2)) + "%")
    print("\n\nPositive tweets:")
    for tweet in ptweets[:10]:
        print(tweet['text'])
        
    print("\n\nNegative tweets:")
    for tweet in ntweets[:10]:
        print(tweet['text'])
        
if __name__ == "__main__": 
    main()

Positive tweets percentage: 28.95%
Negative tweets percentage: 21.05%
Neutral tweets percentage: 50.0%


Positive tweets:
Le modèle américain d’influence mondiale qui vacillait en raison du changement de cap incarné par la présidence de… https://t.co/CjNvmnN3ES
RT @haaohaoo: Scotland's First Minister Nicola Sturgeon admits that "we laugh" at President Donald Trump https://t.co/xk8b7Is6fN
RT @timmehjimmeh: Much like how Walter White and Rick Sanchez are idolised by people who don't realise they are cautionary tales, the DA re…
RT @IAmDawnM: And if Ron says it, it's TRUE.  

Ron Howard Calls Donald Trump a 'Self-Serving, Dishonest, Morally Bankrupt Ego Maniac'
http…
RT @mittdolcino: Tra guerra eterna e fine della storia (di Francis Fukuyama): Pat Buchanan, stimatissimo rappresentante repubblicano USA, a…
RT @IanDunt: Imagine not even being Donald Trump, but a cheap knock-off version of him.
RT @chrislhayes: For all the talk about how Trump is some radical departure from the Good Old GOP,